# Balancer Simulations Math Challenge - Basic Exercises

This is a series of exercises to gain intuition for the core algorithm in Balancer Pools: the Value Function, and invariant V.

**Exercise:** We'll work on Step 1 questions in our working session on May 25 in teams of 2, facilitated by @Octopus and @akrtws.  
**Challenges:** Step 2 will be homework assignments, submitting at least one challenge solved is mandatory for successful participation!  
Here's the notebook with challenges: https://github.com/TokenEngineeringCommunity/BalancerPools_Model/blob/fcb67145e8b0f8a1843fe3c6921dbb5a7085938e/Math%20Challenges-Advanced.ipynb

## Exercise:   
Let's set up a pool.  

We have 100 Token A and 100 Token B, with equal weights.  
The price definition in our pool is constraint by the
Invariant V in Balancer Pools. 

a_bal = balance of token A  
b_bal = balance of token B  
a_weight = weight of token A  
b_weight = weight of token B   

The weights in this pool are equal for both tokens.
For now, we don't have a swap fee.

# Q1.1: 
What's the initial spot price of token A in token B?

In [1]:
#The initial spot price is 1:1, 1 token A equals 1 token B.

In [2]:
import numpy as np
import pandas as pd

import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot

import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# Q1.2:
Now let's assume a series of 99 swaps. With every swap, **1.0 token A is bought from the pool, against token B**.  

**a) Create a table "buy_A"** with

    the token A balances (swap by swap)
    the token B balances (swap by swap) - that are constraint by the value function. 

In [3]:
#set up genesis state
a_bal = 100 # initial balance
b_bal = 100 # initial balance

a_weight = 0.5 #define weight
b_weight = 0.5 #define weight

s_f = 0.0 #swap fee
inv = (a_bal**a_weight)*(b_bal**b_weight) #calculate invariant

a_vals = pd.Series(range(100,0,-1))

#create dataframe with based on a_vals
buy_A = pd.DataFrame(a_vals, columns=['token_A'])

buy_A['invariant'] = inv #value required to calculate token B value

#create values for plot, add Y_balances according to current invariant
buy_A['token_B'] = (buy_A.invariant/(buy_A.token_A**a_weight))**(1/b_weight)# calculate corresponding token_B value according to invariant

buy_A.tail()
# buy_A.shape

,token_A,invariant,token_B
95,5,100.0,2000.000000
96,4,100.0,2500.000000
97,3,100.0,3333.333333
98,2,100.0,5000.000000
99,1,100.0,10000.000000


**b) What do you notice in general?**
By looking at below table, if we go 1 direction of swapping (keep buying token_A):
- For the 1st token_A, we have to pay 1.010101 token_B
- But for the 50th token_A, we have to pay (200-196.078431) ~ 4 token_B

In [4]:
buy_A.iloc[[0,1,49,50],:]

,token_A,invariant,token_B
0,100,100.0,100.000000
1,99,100.0,101.010101
49,51,100.0,196.078431
50,50,100.0,200.000000


**c) How much would Alice have to pay in token B when buying the first 1.0 token A?** <br>
Actual amount Alice has to pay: 1.010101 >  1 (initial spot price)

# Q1.3:

Now let's assume a series of 99 swaps in the opposite direction. We start again with the original state: We have 100 Token A and 100 Token B.   
With every swap, **1.0 token B is bought from the pool, against token A**.

Create a table **'buy_B'** with

    the token A balances (swap by swap)
    the token B balances (swap by swap) - that are constraint by the value function.

In [5]:
b_vals = pd.Series(range(100,0,-1))

buy_B = pd.DataFrame(b_vals, columns=['token_B'])
buy_B['invariant'] = inv 

#create values for plot, add a_balances according to current invariant
buy_B['token_A'] = (buy_B.invariant/(buy_B.token_B**b_weight))**(1/a_weight)# calculate corresponding token A value according to invariant

buy_B.head()

,token_B,invariant,token_A
0,100,100.0,100.000000
1,99,100.0,101.010101
2,98,100.0,102.040816
3,97,100.0,103.092784
4,96,100.0,104.166667


# Q1.4:

a) What are the new balances of token A and token B after 90 swaps token A (in) for B (out)?

After 90 swaps the pool has
- 10 token_B 
- 1000 token_A

In [6]:
buy_B.iloc[[90,91],:]

,token_B,invariant,token_A
90,10,100.0,1000.000000
91,9,100.0,1111.111111


b) How much would Alice have to pay in token A in case she wants to buy the 91st token B? 

``` python
price_of_91st_token_B = (buy_B.iloc[91]['token_A'])-((buy_B.iloc[90]['token_A']))
```

In [7]:
price_of_91st_token_B = (buy_B.iloc[91]['token_A'])-((buy_B.iloc[90]['token_A']))
print(price_of_91st_token_B)

111.11111111111131


# Q1.5:  
Now create a graph (use plotly or similar), and draw the full curve for this series of both kinds of swaps - the AMM curve.

In [8]:
# buy_A = buy_A.drop(columns=['invariant'])
# buy_B = buy_B.drop(columns=['invariant'])

curve = pd.concat([buy_A, buy_B.iloc[::-1]], axis=0, ignore_index=True)

fig = px.line(curve, x="token_A", y="token_B", color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_xaxes(range=[0, 1000])
fig.update_yaxes(range=[0, 1000])
fig.update_layout(height=1000, width=1000, title_text='<b>AMM Curve</b>')

fig.show()

# Q1.6:
Take this plot, and mark 
- the initial price in Q1.1 (starting price)
- the new price in Q1.4 (after 90 swaps)

Side note: Why is it actually not appropriate to call this a "price". What would be a better name?

In [9]:
# buy_A = buy_A.drop(columns=['invariant'])
# buy_B = buy_B.drop(columns=['invariant'])

curve = pd.concat([buy_A, buy_B.iloc[::-1]], axis=0, ignore_index=True)

fig = px.line(curve, x="token_A", y="token_B", color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_xaxes(range=[0, 1200])
fig.update_yaxes(range=[0, 1200])
fig.update_layout(height=1000, width=1000, title_text='<b>AMM Curve with some points</b>')


ini_point = [a_bal,b_bal]
fig.add_trace(go.Scatter(x=[ini_point[0]], y=[ini_point[1]]))
fig.add_annotation(x=100, y=100,
            text="initial balance",
            showarrow=True,)

swap_90_B = [buy_B.token_A[90],buy_B.token_B[90]]     #buy 90 token_B

fig.add_trace(go.Scatter(x=[swap_90_B[0]], y=[swap_90_B[1]]))
fig.add_annotation(x=swap_90_B[0], y=swap_90_B[1],
            text="swap_90_B",
            showarrow=True,)

fig.update_layout(showlegend=False)

fig.show()

# Q1.7:
Formulate a "rule of a thumb", how do swaps effect the price?
- With number of swaps go on one direction, price keeps increasing toward the 'wanted' token as its scarcity increases

## Additional questions:

**Q1:** Buying A) 99 tokens at once vs. B) buying 99x 1 token
Is there a difference in terms of the number of tokens you have to swap in? Is it "more expensive" to swap 99 times?
**Answer: no, there's no difference - you pay the same amount no matter if you buy 1 at a time or 99 at once. Intuition: you are moving on the SAME curve, no matter if you take a huge leap, or walk in small steps.**

**Q2:** Swap fees for buying 99 tokens at once vs. buying 99x 1 token
Again, does it matter in terms of the total amount of tokens (change in balance+ fees) you have to pay?


**Q3:** Price changes and position at the curve
For a relative change in price, does it matter where we are at the curve? ( @mark richardson | bancor explained it very nicely, you might want to watch the recording again, will share it by tomorrow)

**Q4:** Price changes, position at the curve and unequal weights
What's the effect of weights on this? If weights are not equal, and we want to move prices, does it matter where we are at the curve?


### Q1
Buying A) 99 tokens at once vs. B) buying 99x 1 token Is there a difference in terms of the number of tokens you have to swap in? Is it "more expensive" to swap 99 times?

- Invariant V
<div align="center">$V = \prod\limits_{t} {B_t}^{W_{t}}
= a^{w_a}b^{w_b}$</div>

- Number of token B 

<div align="center">
$
b = \left( \frac{V}{a^{w_a}} \right)^{1/w_b}
$
</div>

- **99 token_A at once swap**, meaning only 1 token_A left after that 1 swap, number of token B after that swap is calculated by 
<div align="center">
$
b = \left( \frac{V}{1^{w_a}} \right)^{1/w_b} = V^2
$
</div>

- **99 swaps, 1 token_A each** 
    - Genesis state: 
<div align="center">
    $a_0 = a_{bal}$ <br>
    $b_0 = b_{bal}$
</div>
 
    - 1st swap: 
<div align="center">
    $b_1  = \left( \frac{V}{(a_{bal}-1)^{w_a}} \right)^{1/w_b}$
</div>
  
<div align="center">
    $\Delta b_1 =  b_1 - b_0 =
    \left( \frac{V}{(a_{bal}-1)^{w_a}} \right)^{1/w_b}- 
    \left( \frac{V}{(a_{bal})^{w_a}} \right)^{1/w_b}$
</div>
    
    - 2nd swap: 
<div align="center">
    $b_2 = \left( \frac{V}{(a_{bal}-2)^{w_a}} \right)^{1/w_b}$
</div>
  
<div align="center">
    $\Delta b_2 = b_2 - b_1=
    \left( \frac{V}{(a_{bal}-2)^{w_a}} \right)^{1/w_b}- 
    \left( \frac{V}{(a_{bal}-1)^{w_a}} \right)^{1/w_b}$
</div>

...

    - 99th swap: 
<div align="center">
    $b_{99} = \left( \frac{V}{(a_{bal}-99)^{w_a}} \right)^{1/w_b}$
</div>
  
<div align="center">
    $\Delta b_{99} =  b_{99} - b_{98} =
    \left( \frac{V}{(a_{bal}-99)^{w_a}} \right)^{1/w_b}- 
    \left( \frac{V}{(a_{bal}-98)^{w_a}} \right)^{1/w_b}$
</div>

    - After 99 swaps
<div align="center">
$
b_{99} = b_0 + \sum_{i=1}^{99} \Delta b =
\left( \frac{V}{(a_{bal}-99)^{w_a}} \right)^{1/w_b} = V^2
$	
</div>


**Generalize the problem: with the given value of number of token_A in the pool, there will be only one value for number of token_B that fulfills invariant V requirement, regardless of how many swaps to come to that state**

# Now, let's consider weights!

We continue with the value function V = a^w_a*b^w_b  
where  
a = balance of token asset A  
b = balancer of token asset B  
w_a = weight of token asset A  
w_b = weight of token asset B  

# Q2.1:  
Write down the value function for the pool in Q1.1!

**Invariant V**
<div align="center">$V = \prod\limits_{t} {B_t}^{W_{t}}
= a^{w_a}b^{w_b}$</div>

# Q2.2:
Let's got back to your initial balances in Step 1 in the pool:  
100 tokens A  
100 tokens B  

How do you need to change the weights in order to land at a **price of 4 tokens A : 1 token B**

***Answer***
 <div align="center">
    $
SP = \frac{\frac{a_{bal}}{w_a}}{\frac{b_{bal}}{w_b}}
    $
</div>

**4 tokens A : 1 token B** <br>

=> SP = 4

=> $ \frac{w_b}{w_a} = 4 $

with $w_a + w_b = 1$ 

=> $w_a =0.2$, $w_b =0.8$


Provide the new value function!

# Q2.3:
Create a graph showing the new AMM Curve in Q2.2  
Compare to the graph in Q1.4 - how does a change in weights change the graph?

In [10]:
import matplotlib
from matplotlib import pyplot as plt

In [11]:
#set up genesis state
a_bal = 100 # initial balance
b_bal = 100 # initial balance

a_weight = 0.2 #define weight
b_weight = 0.8 #define weight

s_f = 0.0 #swap fee
inv = (a_bal**a_weight)*(b_bal**b_weight) #calculate invariant

b_vals = pd.Series(range(100,0,-1))

buy_B_w1 = pd.DataFrame(b_vals, columns=['token_B'])
buy_B_w1['invariant'] = inv 

#create values for plot, add a_balances according to current invariant
buy_B_w1['token_A'] = (buy_B_w1.invariant/(buy_B_w1.token_B**b_weight))**(1/a_weight)# calculate corresponding token A value according to invariant

buy_B_w1.head()

,token_B,invariant,token_A
0,100,100.0,100.000000
1,99,100.0,104.102036
2,98,100.0,108.416578
3,97,100.0,112.956977
4,96,100.0,117.737570


In [12]:
#set up genesis state
a_bal = 100 # initial balance
b_bal = 100 # initial balance

a_weight = 0.2 #define weight
b_weight = 0.8 #define weight

s_f = 0.0 #swap fee
inv = (a_bal**a_weight)*(b_bal**b_weight) #calculate invariant

a_vals = pd.Series(range(100,0,-1))
buy_A_w1 = pd.DataFrame(a_vals, columns=['token_A'])
buy_A_w1['invariant'] = inv #value required to calculate token B value
buy_A_w1['token_B'] = (buy_A_w1.invariant/(buy_A_w1.token_A**a_weight))**(1/b_weight)# calculate corresponding token_B value according to invariant

b_vals = pd.Series(range(100,0,-1))
buy_B_w1 = pd.DataFrame(b_vals, columns=['token_B'])
buy_B_w1['invariant'] = inv 
buy_B_w1['token_A'] = (buy_B_w1.invariant/(buy_B_w1.token_B**b_weight))**(1/a_weight)# calculate corresponding token A value according to invariant

buy_A_w1 = buy_A_w1.drop(columns=['invariant'])
buy_B_w1 = buy_B_w1.drop(columns=['invariant'])
curve_w1 = pd.concat([buy_A_w1, buy_B_w1.iloc[::-1]], axis=0, ignore_index=True)

In [13]:
curve.iloc[95:105,:]
# curve.iloc[90:110,:]

,token_A,invariant,token_B
95,5.000000,100.0,2000.000000
96,4.000000,100.0,2500.000000
97,3.000000,100.0,3333.333333
98,2.000000,100.0,5000.000000
99,1.000000,100.0,10000.000000
100,10000.000000,100.0,1.000000
101,5000.000000,100.0,2.000000
102,3333.333333,100.0,3.000000
103,2500.000000,100.0,4.000000
104,2000.000000,100.0,5.000000


In [14]:
curve

,token_A,invariant,token_B
0,100.000000,100.0,100.000000
1,99.000000,100.0,101.010101
2,98.000000,100.0,102.040816
3,97.000000,100.0,103.092784
4,96.000000,100.0,104.166667
...,...,...,...
195,104.166667,100.0,96.000000
196,103.092784,100.0,97.000000
197,102.040816,100.0,98.000000
198,101.010101,100.0,99.000000


In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=curve["token_A"], y=curve["token_B"],
                    mode='lines',
                    name='50-50'))
fig.add_trace(go.Scatter(x=curve_w1["token_A"], y=curve_w1["token_B"],
                    mode='lines',
                    name='20A - 80B'))


fig.update_xaxes(range=[0, 300])
fig.update_yaxes(range=[0, 300])
fig.update_layout(height=1000, width=1000, title_text='<b>AMM Curves</b>')


# fig.update_layout(showlegend=False) 

fig.show()

# Q2.4:
Compare token prices in this pool.  
How much would Alice have to pay in case there are only 2 tokens left in the pool  
    **a) buy 1.0 token A for token B**
    
``` python
price_of_99st_token_A = (curve.iloc[99]['token_B'])-((curve.iloc[98]['token_B']))
```
    
   **b) buy 1.0 token B for token A**   
```python
price_of_99st_token_B = (curve.iloc[100]['token_A'])-((curve.iloc[101]['token_A']))
```

In [16]:
print("Equal weights:")
print('price_of_99st_token_A: {}'.format(curve.iloc[99]['token_B']-(curve.iloc[98]['token_B'])))
print('price_of_99st_token_B: {}'.format(curve.iloc[100]['token_A']-(curve.iloc[101]['token_A'])))
print('\n')

print("0.2 A - 0.8 B")
print('price_of_99st_token_A: {}'.format(curve_w1.iloc[99]['token_B']-(curve_w1.iloc[98]['token_B'])))
print('price_of_99st_token_B: {}'.format(curve_w1.iloc[100]['token_A']-(curve_w1.iloc[101]['token_A'])))

Equal weights:
price_of_99st_token_A: 5000.000000000002
price_of_99st_token_B: 5000.000000000002


0.2 A - 0.8 B
price_of_99st_token_A: 50.312971169588536
price_of_99st_token_B: 9375000000.000011
